In [27]:
import pickle
import numpy as np
from processing.text_processing import *
import random
import sys
try:
    import pyLDAvis
except:
# Install a pip package in the current Jupyter kernel
    !{sys.executable} -m pip install pyLDAvis
    import pyLDAvis
pyLDAvis.enable_notebook()
import pyLDAvis.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [24]:
with open('./selected_3topics_qs.pkl', "rb") as file:
    collection = pickle.load(file )

In [25]:
collection[0:5]

['Why does velocity affect time ? Does velocity affect space geometry ?',
 'What is the cause of magnetic force in magnet ?',
 'How does the electrostatic force f revolving electron becomes equal to the centripetal force ?',
 'How fast will humans die If the sun suddenly turn off and solidify maintaining the same gravitational force ?',
 "How can a spacecraft get momentum from their trusters if there 's nothing pushing back on what comes out of their trusters ?"]

In [26]:
random.shuffle(collection)
collection[0:5]

['How can dB/dt be defined as a function of the velocity of a permanent magnet ( with a known B value ) moving towards and then through a conducting loop ?',
 'Is Ludwig Boltzmann entropy law considered a mathematical improbability even though it is considered a statistical fact ?',
 'The initial velocity of a stone thrown vertically upwards to reach a height of 10 meters is ?',
 'I ’ m 14 years old and 5 ’ 2 ” , I have only grown like half an inch in the last year. Am I done growing ? My mom is 5 ’ 4 ” and my dad is 6 ’ .',
 "We humans are destroying our own planet by capitalism. I believe we 're like termites. We eat all the resources and then move somewhere else. What do you think ?"]

In [9]:
question_cls = Corpus(collection, 
                      lowercase=True, 
                      remove_punct=True, 
                      remove_stopwords=True, 
                      lemmatize=True)

The corpus contains 136708 total tokens where 15669 are unique


Below is inspired by pyLDAvis repo example

In [10]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(collection)
print(dtm_tf.shape)

(16993, 2013)


In [13]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(collection)
print(dtm_tfidf.shape)

/Users/alistar/Library/Python/3.7/lib/python/site-packages/sklearn/feature_extraction/text.py:1817: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  UserWarning)


(16993, 2013)


In [14]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=5, random_state=0)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=5, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [28]:
tf_vectorizer
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.208562  0.110441       1        1  21.942054
4      0.059408 -0.213341       2        1  21.363360
2     -0.201080 -0.120978       3        1  21.313912
0      0.148090  0.037026       4        1  18.861289
1     -0.214980  0.186851       5        1  16.519385, topic_info=     Category         Freq        Term        Total  loglift  logprob
724   Default  3312.000000       force  3312.000000  30.0000  30.0000
1339  Default  1580.000000       plant  1580.000000  29.0000  29.0000
810   Default  1756.000000       green  1756.000000  28.0000  28.0000
1335  Default  1757.000000      planet  1757.000000  27.0000  27.0000
1705  Default  1451.000000        star  1451.000000  26.0000  26.0000
...       ...          ...         ...          ...      ...      ...
1711   Topic5    59.007169       state   103.104665   1.2425  -5.5308
216    Topic5    50.462128       build    68.138751   1.5003  -5.6872
492    Topic5    67.309978  difference   234.766875   0.5514  -5.3992
1910   Topic5    60.110659       using   187.027230   0.6656  -5.5123
724    Topic5    65.236779       force  3312.586884  -2.1268  -5.4304

[314 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
10        1  0.723313  acceleration
10        2  0.003837  acceleration
10        4  0.272442  acceleration
18        2  0.980226        acting
18        4  0.018152        acting
...     ...       ...           ...
2005      1  0.773772         years
2005      2  0.139375         years
2005      3  0.086509         years
2006      3  0.992042        yellow
2011      1  0.983667          zero

[460 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 3, 1, 2])